In [27]:
# Ujian Tengah Semester STKI  
## Mini Search Engine – Term Weighting, VSM, Boolean, Evaluasi  
### Nama: IJLAL FACHRY ATTALLAH 
### NIM : A11.2023.15170

In [28]:
import sys
import os

# masuk ke folder project (1 level ke atas dari notebooks/)
project_root = os.path.abspath("..")
sys.path.append(project_root)

project_root


'd:\\KULIAH\\Semester 5\\STKI\\stki-uts-A11.2023.15170-Ijlal Fachry Attallah'

In [29]:
import pandas as pd
import numpy as np

from src.preprocess import clean_text
from src.boolean_ir import BooleanIR
from src.vsm_ir import VSM
from src.search import SearchEngine
from src.eval import (
    precision_recall_f1,
    map_at_k,
    ndcg_at_k
)


In [30]:
food_df = pd.read_csv(r"d:/KULIAH/Semester 5/STKI/stki-uts-A11.2023.15170-Ijlal Fachry Attallah/data/1662574418893344.csv")
ratings_df = pd.read_csv(r"d:/KULIAH/Semester 5/STKI/stki-uts-A11.2023.15170-Ijlal Fachry Attallah/data/ratings.csv")


df = pd.merge(food_df, ratings_df, on="Food_ID", how="inner")
df.head()


,Food_ID,Name,C_Type,Veg_Non,Describe,User_ID,Rating
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin...",49.0,5.0
1,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin...",71.0,10.0
2,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni...",9.0,3.0
3,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni...",22.0,5.0
4,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni...",39.0,10.0


In [31]:
df["clean_desc"] = df["Describe"].apply(clean_text)
df[["Describe", "clean_desc"]].head()

,Describe,clean_desc
0,"white balsamic vinegar, lemon juice, lemon rin...",white balsamic vinegar lemon juice lemon rind ...
1,"white balsamic vinegar, lemon juice, lemon rin...",white balsamic vinegar lemon juice lemon rind ...
2,"olive oil, chicken mince, garlic (minced), oni...",olive oil chicken mince garlic minced onion sa...
3,"olive oil, chicken mince, garlic (minced), oni...",olive oil chicken mince garlic minced onion sa...
4,"olive oil, chicken mince, garlic (minced), oni...",olive oil chicken mince garlic minced onion sa...


In [32]:
boolean_model = BooleanIR(df["clean_desc"].tolist())


In [33]:
vsm_model = VSM(df["clean_desc"].tolist())


In [34]:
query = "summer squash salad"

results_boolean = boolean_model.search(query)
results_boolean[:10]


[]

In [35]:
results_vsm = vsm_model.search(query, k=10)
results_vsm

(array([  0,   1, 156, 155, 501, 500, 499, 498, 497, 496]),
 array([0.45567061, 0.45567061, 0.10927012, 0.10927012, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]))

In [36]:
engine = SearchEngine(model_type="vsm")

query = "summer squash salad"
results = engine.search(query, k=5)
results


[(np.int64(387), np.float64(0.4669640746739993)),
 (np.int64(0), np.float64(0.4280560229362077)),
 (np.int64(309), np.float64(0.29874894826450116)),
 (np.int64(376), np.float64(0.20221313740882046)),
 (np.int64(51), np.float64(0.09697445779827812))]

In [37]:
engine = SearchEngine(model_type="boolean")

query = "summer squash salad"
results = engine.search(query, k=5)
results

[]

In [38]:
gold_set = {
    "summer squash salad": [12, 55, 203, 17]   # ID dokumen relevan
}
gold_set


{'summer squash salad': [12, 55, 203, 17]}

In [39]:
# Jalankan VSM
top_docs, scores = vsm_model.search(query, k=10)

# Predicted = list doc_id
predicted = top_docs

# Hitung precision, recall, F1
p, r, f1 = precision_recall_f1(gold_set[query], predicted)

p, r, f1


(0.0, 0.0, 0)

In [40]:
map_score = map_at_k(gold_set[query], predicted, k=10)
ndcg_score = ndcg_at_k(gold_set[query], predicted, k=10)

map_score, ndcg_score


(0.0, np.float64(0.0))

In [41]:
comparison = {
    "boolean_top5": results_boolean[:5],
    "vsm_top5": results_vsm[:5]
}

comparison


{'boolean_top5': [],
 'vsm_top5': (array([  0,   1, 156, 155, 501, 500, 499, 498, 497, 496]),
  array([0.45567061, 0.45567061, 0.10927012, 0.10927012, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]))}